In [ ]:
import Preprocess_Data as pdata
import pandas as pd
import pickle

In [ ]:
glen_H_data_raw = pd.read_csv("Labeled_glen_2017-03-19_to_2017-03-23_simplified.csv", index_col=0)
glen_H_data_raw.head()

In [ ]:
glen_H_data = pdata.Preprocess_Data(glen_H_data_raw)

In [ ]:
glen_H_data.Norm()
data = glen_H_data.Feature_additions()
data, X, Y = glen_H_data.load_data_train()
print "DATA:\t", data.head()
print "X", X
print "Y", Y

In [ ]:
data.groupby("classification").size()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_Class = RandomForestClassifier(bootstrap=True,min_samples_leaf=1,n_estimators=1500,\
                                  min_samples_split=2, criterion='gini', max_features=12, max_depth=10)
RF_Class.fit(X,Y)

In [ ]:
import pickle
pickle_glen_C_032617 = pickle.dump(RF_Class, open("pickle_glen_C_032617.p", "wb")) 

In [ ]:
emil_H_data_raw = pd.read_csv("Labeled_Test_Data_Emil_03_25_2017.csv", index_col=0)
emil_H_data_raw.groupby("classification").size()
len(emil_H_data_raw)

In [ ]:
emil_H_data = Preprocess_Data(emil_H_data_raw)

In [ ]:
emil_H_data.Norm()
data_E = emil_H_data.Feature_additions()
data_E, X_E, Y_E = emil_H_data.load_data()

# Emil Testing

In [21]:
import Preprocess_Data as pdata
import pandas as pd
import numpy as np
import pickle
import db.knowhere_db as kdb
reader = kdb.Reader(db_name='knowhere')


In [2]:
pkl = pickle.load(open("D:/drive/emilparikh/Professional/School/NYC Data Science Academy/class/projects/Capstone/Knowhere/app/data/pickle_glen_C_032617.p", "rb"))

In [32]:
glen24th = reader.get_dataframe_pivoted(collection="iphone", username="glen", sensor=["Acceleration", "Magnetometer"], commute=True, 
                                min_date="2017-03-24 00:00:00", max_date="2017-03-25 00:00:00")

glen24th = pdata.Preprocess_Data(glen24th)
glen24th.Norm()
data = glen24th.Feature_additions()
X = glen24th.load_data_test()
pred = np.vectorize(lambda x: activity_labels[x])(pkl.predict(X))

In [13]:
activity_labels = {0:"driving", 1:"elevator", 2:"standing", 3:"train", 4:"train", 5:"walking"}

In [102]:
df=pd.DataFrame({"date":data.index, "label":pred})
df["hm"] = df["date"].apply(lambda x: "{0}{1}".format(x.hour, x.minute))

In [103]:
df = df.groupby(["label", "hm"]).agg({"date": lambda i: (max(i)-min(i)).total_seconds()})

In [104]:
# date actually = seconds.
df = df.reset_index().groupby("label").sum().reset_index()
df.columns = ["label", "seconds"]

In [124]:
def set_percents(x):
    activity_percents[x.label] = round(x.seconds/total_seconds*100,2)

In [125]:
total_seconds = np.sum(df.seconds)
activity_percents = {}

df.apply(set_percents, axis=1)

activity_percents

{'driving': 12.76,
 'elevator': 0.4,
 'standing': 0.0,
 'train': 35.89,
 'walking': 50.96}